In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

import torch.optim as optim
import copy
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torchvision.transforms.functional import to_pil_image
from torchvision import datasets
from matplotlib.pyplot import imshow
from sklearn.metrics import f1_score
from torch.optim import SGD, Adam

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

c:\Python310\lib\site-packages\sklearn\utils\_param_validation.py:11: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.3)
  from scipy.sparse import csr_matrix, issparse


cpu


In [2]:
transform = transforms.Compose(
    [
        transforms.Resize(size=(180, 180)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

train_dir = "./data/train"  # path to the train folder
validation_dir = "./data/validation"  # path to the validation folder
test_dir = "./data/test"  # path to test folder

train_data = datasets.ImageFolder(root=train_dir, transform=transform)

validation_data = datasets.ImageFolder(root=validation_dir, transform=transform)

test_data = datasets.ImageFolder(root=test_dir, transform=transform)

print(
    f"Train data:\n{train_data}\n\nValidation data:\n{validation_data}\n\nTest data:\n{test_data}"
)

data_loaders = {
    "train": DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2),
    "validation": DataLoader(
        validation_data, batch_size=4, shuffle=True, num_workers=2
    ),
}

Train data:
Dataset ImageFolder
    Number of datapoints: 3689
    Root location: ./data/train
    StandardTransform
Transform: Compose(
               Resize(size=(180, 180), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

Validation data:
Dataset ImageFolder
    Number of datapoints: 495
    Root location: ./data/validation
    StandardTransform
Transform: Compose(
               Resize(size=(180, 180), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

Test data:
Dataset ImageFolder
    Number of datapoints: 494
    Root location: ./data/test
    StandardTransform
Transform: Compose(
               Resize(size=(180, 180), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.

In [4]:
from models.modelv3 import SimpleCNN3

model = SimpleCNN3()

In [5]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    train_loss, correct = 0, 0
    size = len(dataloader.dataset)

    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    accuracy = 100 * correct / size
    return train_loss / len(dataloader), accuracy


def validate(dataloader, model, loss_fn):
    model.eval()
    validation_loss, correct = 0, 0
    size = len(dataloader.dataset)

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            validation_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    accuracy = 100 * correct / size
    return validation_loss / len(dataloader), accuracy

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizers = {
    "SGD": lambda params: SGD(params, lr=0.001),
    "Adam": lambda params: Adam(params, lr=0.001),
}
epochs = 3

In [ ]:
results = {"optimizer": [], "train_acc": [], "validation_acc": []}

for optimizer_name, optimizer_fn in optimizers.items():
    print(f"Training with optimizer: {optimizer_name}")
    model = SimpleCNN().to(device)  # Reinitialize model for each optimizer
    optimizer = optimizer_fn(model.parameters())

    train_accuracies = []
    validation_accuracies = []

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        train_loss, train_acc = train(data_loaders["train"], model, loss_fn, optimizer)
        val_loss, val_acc = validate(data_loaders["validation"], model, loss_fn)

        train_accuracies.append(train_acc)
        validation_accuracies.append(val_acc)

        print(f"Train Accuracy: {train_acc:.2f}%, Validation Accuracy: {val_acc:.2f}%")

    results["optimizer"].append(optimizer_name)
    results["train_acc"].append(train_accuracies[-1])  # Final train accuracy
    results["validation_acc"].append(
        validation_accuracies[-1]
    )  # Final validation accuracy

Training with learning rate: 1e-05
Epoch 1/3
 Training Accuracy: 69.31%, Training Loss: 569.4357
 Validation Accuracy: 69.49%, Validation Loss: 75.0760
Train Accuracy: 69.31%, Validation Accuracy: 69.49%
Epoch 2/3
 Training Accuracy: 69.88%, Training Loss: 554.5916
 Validation Accuracy: 69.49%, Validation Loss: 74.5370
Train Accuracy: 69.88%, Validation Accuracy: 69.49%
Epoch 3/3
 Training Accuracy: 69.88%, Training Loss: 546.2582
 Validation Accuracy: 69.49%, Validation Loss: 74.0367
Train Accuracy: 69.88%, Validation Accuracy: 69.49%
Last Train Accuracy for LR=1e-05: 69.88%
Training with learning rate: 0.0001
Epoch 1/3


In [ ]:
# Plot results
plt.figure(figsize=(10, 5))

x = range(len(results["optimizer"]))

# Train accuracy
plt.bar(x, results["train_acc"], width=0.4, label="Train Accuracy", align="center")
# Validation accuracy
plt.bar(
    [i + 0.4 for i in x],
    results["validation_acc"],
    width=0.4,
    label="Validation Accuracy",
    align="center",
)

plt.xticks([i + 0.2 for i in x], results["optimizer"])
plt.title("Optimizer Comparison: Train vs Validation Accuracy")
plt.xlabel("Optimizer")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.tight_layout()
plt.show()